# Muhammad Zain Ali
# 22i-0562
# Lab 8
# Artificial Intelligence

## Greedy Best First Search

In [16]:
from queue import PriorityQueue
import random

class State:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

class Node:
    def __init__(self, state, heuristic_cost):
        self.state = state
        self.heuristic_cost = heuristic_cost

    def __lt__(self, other):
        return self.heuristic_cost < other.heuristic_cost

def is_valid(x, y, size):
    return 0 <= x < size and 0 <= y < size

def manhattan_distance(state, goal):
    return abs(state.x - goal.x) + abs(state.y - goal.y)

def best_first_search(start, goal, forest_grid):
    size = len(forest_grid)
    visited = set()
    PQ = PriorityQueue()
    PQ.put(Node(start, manhattan_distance(start, goal)))

    while not PQ.empty():
        current_node = PQ.get()
        current_state = current_node.state

        if current_state == goal:
            return current_node

        if (current_state.x, current_state.y) in visited:
            continue

        visited.add((current_state.x, current_state.y))

        x, y = current_state.x, current_state.y
        for dx, dy in [(0, 1), (-1, 0), (-1, 1)]:
            new_x, new_y = x + dx, y + dy

            if is_valid(new_x, new_y, size):
                new_state = State(new_x, new_y, current_node)

                PQ.put(Node(new_state, manhattan_distance(new_state, goal)))

    return None

def print_path_and_actions(path):
    print("Path:")
    for i, (x, y) in enumerate(reversed(path)):
        print(f"({x}, {y})", end=" ")
        if (i + 1) % 5 == 0:
            print()
    actions = [(abs(path[i][0] - path[i-1][0]), abs(path[i][1] - path[i-1][1])) for i in range(1, len(path))]
    total_cost = sum(sum(action) for action in actions)
    print("\n__________________________________")
    print("Total cost of the path:", total_cost)
    print("__________________________________")
    print("Grid representation:")

def print_grid_with_path(forest_grid, path):
    size = len(forest_grid)
    for i in range(size):
        for j in range(size):
            if (i, j) in path:
                print(" *", end=" ")
            else:
                print(f"{forest_grid[i][j]:2}", end=" ")
        print()

class ForestGrid:
    def __init__(self, size=15):
        self.size = size
        self.grid = self.generate_grid(size)

    def generate_grid(self, size):
        grid = []
        for i in range(size):
            row = []
            for j in range(size):
                cell_type = random.choice([0, 0, 0, 0, 0, 1, 1])
                row.append(cell_type)
            grid.append(row)
        start_x, start_y = 14, 0
        goal_x, goal_y = 0, 14
        grid[start_x][start_y] = 2
        grid[goal_x][goal_y] = 3
        return grid

    def print_grid(self):
        print("__________________________________")
        for row in self.grid:
            for cell in row:
                print(f"{cell:2}", end=" ")
            print()
        print("__________________________________")

def main():
    size = 15
    forest = ForestGrid(size)
    print("Generated Forest Grid:")
    forest.print_grid()

    start_state = State(14, 0)
    goal_state = State(0, 14)

    result = best_first_search(start_state, goal_state, forest.grid)

    if result is not None:
        path = []
        node = result
        while node is not None:
            path.append((node.state.x, node.state.y))
            node = node.state.parent
        
        print_path_and_actions(path)
        print_grid_with_path(forest.grid, path)
    else:
        print("Goal is unreachable.")

if __name__ == "__main__":
    main()


Generated Forest Grid:
__________________________________
 0  0  0  0  1  0  1  0  1  0  0  1  0  0  3 
 1  1  0  0  0  1  0  0  0  0  1  1  0  1  1 
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 
 0  1  1  0  0  0  0  0  0  1  1  1  1  1  1 
 0  0  0  1  0  1  0  0  1  1  0  0  0  0  0 
 0  1  0  0  0  1  0  1  0  1  0  0  0  0  0 
 0  0  0  0  0  0  0  0  0  1  1  0  0  0  0 
 0  1  0  0  0  0  0  0  0  1  0  0  1  0  1 
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  1 
 0  0  0  1  0  1  0  1  1  0  1  1  1  0  0 
 0  0  0  0  0  0  0  0  0  0  1  0  0  1  1 
 0  1  0  1  0  0  0  0  0  0  0  0  0  0  0 
 0  1  1  0  0  0  1  0  0  0  0  0  0  1  1 
 0  0  0  0  1  0  1  0  0  1  0  1  1  0  0 
 2  0  0  0  1  1  0  0  0  1  0  0  1  0  1 
__________________________________
Path:
(14, 0) (13, 1) (12, 2) (11, 3) (10, 4) 
(9, 5) (8, 6) (7, 7) (6, 8) (5, 9) 
(4, 10) (3, 11) (2, 12) (1, 13) (0, 14) 

__________________________________
Total cost of the path: 28
______________________________

## A_Star Search

In [1]:
from queue import PriorityQueue
import random

class State:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

class Node:
    def __init__(self, state, path_cost, heuristic_cost):
        self.state = state
        self.path_cost = path_cost
        self.heuristic_cost = heuristic_cost

    def __lt__(self, other):
        return (self.path_cost + self.heuristic_cost) < (other.path_cost + other.heuristic_cost)

def is_valid(x, y, size):
    return 0 <= x < size and 0 <= y < size

def manhattan_distance(state, goal):
    return abs(state.x - goal.x) + abs(state.y - goal.y)

def a_star(start, goal, forest_grid):
    size = len(forest_grid)
    visited = set()
    PQ = PriorityQueue()
    PQ.put(Node(start, 0, manhattan_distance(start, goal)))

    while not PQ.empty():
        current_node = PQ.get()
        current_state = current_node.state

        if current_state == goal:
            return current_node

        if (current_state.x, current_state.y) in visited:
            continue

        visited.add((current_state.x, current_state.y))

        x, y = current_state.x, current_state.y
        for dx, dy in [(0, 1), (-1, 0), (-1, 1)]:
            new_x, new_y = x + dx, y + dy

            if is_valid(new_x, new_y, size):
                new_state = State(new_x, new_y, current_node)

                if (dx, dy) == (0, 1):
                    new_cost = current_node.path_cost + 2
                elif (dx, dy) == (-1, 0):
                    new_cost = current_node.path_cost + 2
                elif (dx, dy) == (-1, 1):
                    new_cost = current_node.path_cost + 3

                PQ.put(Node(new_state, new_cost, manhattan_distance(new_state, goal)))

    return None

def print_path_and_actions(path):
    print("Path:")
    for i, (x, y) in enumerate(reversed(path)):
        print(f"({x}, {y})", end=" ")
        if (i + 1) % 5 == 0:
            print()
    actions = [(abs(path[i][0] - path[i-1][0]), abs(path[i][1] - path[i-1][1])) for i in range(1, len(path))]
    total_cost = sum(sum(action) for action in actions)
    print("\n__________________________________")
    print("Total cost of the path:", total_cost)
    print("__________________________________")
    print("Grid representation:")

def print_grid_with_path(forest_grid, path):
    size = len(forest_grid)
    for i in range(size):
        for j in range(size):
            if (i, j) in path:
                print(" *", end=" ")
            else:
                print(f"{forest_grid[i][j]:2}", end=" ")
        print()

class ForestGrid:
    def __init__(self, size=15):
        self.size = size
        self.grid = self.generate_grid(size)

    def generate_grid(self, size):
        grid = []
        for i in range(size):
            row = []
            for j in range(size):
                cell_type = random.choice([0, 0, 0, 0, 0, 1, 1])
                row.append(cell_type)
            grid.append(row)
        start_x, start_y = 14, 0
        goal_x, goal_y = 0, 14
        grid[start_x][start_y] = 2
        grid[goal_x][goal_y] = 3
        return grid

    def print_grid(self):
        print("__________________________________")
        for row in self.grid:
            for cell in row:
                print(f"{cell:2}", end=" ")
            print()
        print("__________________________________")

def main():
    size = 15
    forest = ForestGrid(size)
    print("Generated Forest Grid:")
    forest.print_grid()

    start_state = State(14, 0) # northwest corner
    goal_state = State(0, 14) # southeast corner

    result = a_star(start_state, goal_state, forest.grid)

    if result is not None:
        path = []
        node = result
        while node is not None:
            path.append((node.state.x, node.state.y))
            node = node.state.parent
        
        print_path_and_actions(path)
        print_grid_with_path(forest.grid, path)
    else:
        print("Goal is unreachable.")

if __name__ == "__main__":
    main()


Generated Forest Grid:
__________________________________
 0  1  1  1  1  0  0  0  0  0  1  0  0  1  3 
 0  0  1  0  0  0  1  0  0  0  0  1  0  0  0 
 1  1  0  0  0  0  1  0  1  0  0  0  0  1  1 
 0  0  0  0  0  0  1  0  0  1  0  0  0  0  1 
 0  0  1  0  0  0  0  0  1  1  0  1  1  0  0 
 1  1  1  1  1  0  0  0  0  0  0  1  0  0  1 
 0  0  0  0  0  0  1  0  0  1  0  0  1  1  0 
 1  0  0  1  0  1  1  0  1  0  0  1  0  0  1 
 0  0  1  1  0  0  1  0  0  1  1  0  1  0  0 
 1  0  1  0  1  0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  1  0  0  0  1  1  1  0  0  0  0 
 1  1  0  0  0  0  0  1  0  0  0  0  0  0  0 
 1  0  0  0  0  0  0  0  1  1  0  0  1  1  1 
 1  0  0  0  0  0  0  0  0  1  1  0  1  0  0 
 2  0  0  0  0  1  0  0  1  0  1  0  1  0  0 
__________________________________
Path:
(14, 0) (13, 1) (12, 2) (11, 3) (10, 4) 
(9, 5) (8, 6) (7, 7) (6, 8) (5, 9) 
(4, 10) (3, 11) (2, 12) (1, 13) (0, 14) 

__________________________________
Total cost of the path: 28
______________________________